#Demonstration of atomman.lammps code design

__The atomman.lammps module has a number of tools for interactions with the LAMMPS molecular dynamics program. The principle actions of atomman.lammps are:__

1. Generating the LAMMPS input script lines associated with a given interatomic potential.

2. Generating the LAMMPS input script lines associated with creating a new system. 

3. Reading and writing system information to and from LAMMPS data and dump files.

4. Reading the standard output or log.lammps file and parsing out the thermo data.

__These actions allow for the integration of LAMMPS simulations into a Python framework to facilitate rapid development of new simulations and easy adaption to different potentials or material systems.__

##0. Initial Setup

###0.1 Python package imports

In [1]:
#Standard Python Libraries
import subprocess
import os

import numpy as np

#Custom package
import atomman

###0.2 LAMMPS Executable

__Running LAMMPS simulations requires having a LAMMPS executable accessible.  This needs to be specified for your system.__

In [2]:
#Specify which LAMMPS executible to use
lammps_exe = 'C:/Users/lmh1/Documents/lmp_serial.exe'

#Test that lammps_exe is a file
if not os.path.isfile(lammps_exe):
    raise ValueError('Could not find LAMMPS executable ' + lammps_exe)

##1.  Representing a LAMMPS potential

__The LAMMPS molecular dynamics program is capable of performing simulations and calculations using a wide variety of interatomic potentials.  To facilitate this, the LAMMPS input scripts allow for the specification of numerous parameters that define or influence the interatomic interaction models.  Across potentials, the values, types and numbers of necessary parameters may drastically vary.  However, an implementation of a potential into LAMMPS is associated with a particular set of parameters that fully defines that atomistic model.__

__The parameter set associated with running LAMMPS with a particular potential implementation can be collected together into a data model.  For AtomMan, a json/XML based data model is used that is capable of fully representing the broad spectrum of interatomic potential parameters in a format that is easily accessible both for human and machine readability. A class atomman.lammps.Potential reads in data model files and handles all conversions, etc. using methods.  This Potential class, along with it being decoupled from System, allows for different interatomic potentials to be easily assigned and exchanged.__

##1.1 Data model format

__This section outlines the various fields in the data model and what they mean.__

__"potentialID"__ lists the identifiers associated with this particular potential instance. 

- "descriptionIdentifier" is used to give the potential instance a name.  It should match the file name.

__"units"__ specifies the LAMMPS units type.

__"atom_style"__ specifies the LAMMPS atom_style type.

__"atom"__ lists the element types used by this potential, and any other parameters specific to that element type. 

- "element" specifies the name of the element (using the standard symbol tag).

- "symbol" specifies the unique tag for the element to associate with atomic interactions.  This often is equivalent to "element", but allows for multiple models to exist for the same element (e.g. Si1, Si2, etc.).  If "symbol" is not listed, its value is set to be equal to "element"'s value.

- "mass" is the atomic mass to be used.  If "mass" is not listed, its value is set to the standard value associated with "element".

- Other per-atom type properties can also be specified, such as "charge".  These are supported by the data model, but not fully integrated into AtomMan yet...

__"pair_style"__ specifies the LAMMPS pair_style type and any associated terms.

- "type" is the pair_style type.

- "term" is a list of any additional terms to include on the pair_style line of the LAMMPS input file. The choices for terms are listed below.

__"pair_coeff"__ lists the interaction parameters associated with this potential implementation.  The order of terms is retained allowing for hybrid styles to be used.  

- "interaction" specifies the element symbols associated with specific interaction terms.  For pair potentials, this is a list of the two atom symbols of a pair interaction.  For many-body potentials, this is a list of all unique atom symbols associated with the interaction model.  If "interaction" is not listed, then the following terms are assumed to be associated with all atom symbols.

- "term" is a list of all terms of an interaction set to include on a pair_coeff line of the LAMMPS input file. The choices for terms are listed below. 

Both __"pair_style"__ and __"pair_coeff"__ use the same convention for the __"term"__ choices.  Each choice is appended in the same order as they are listed in the data model.  The choices are:

- "option" is a value or string that is appended on.

- "file" is a file name associated with the interaction model, e.g. an EAM setfl file.

- "symbols": True indicates that the potential style requires a list of the element symbols associated with the System's atypes.

- "symbolsList": True indicates that the potential style requires a list of all unique element symbols associated with the interaction.  This is needed for some pair styles that use complex library files, such as MEAM. 



In [3]:
print """{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "pot-name"
        },
        "units": "units",
        "atom_style": "style",
        "atom": {
            "element": "El",
            "symbol": "El1",
            "mass": 1.00
        },     
        "pair_style": {
            "type": "pair",
            "term": {
                "option": 1.0
            }
        },
        "pair_coeff": {
            "interaction": {
                "element": [
                    "El",
                    "El"
                ]
            },
            "term": [
                {
                    "option": 1.0
                },
                {
                    "symbolsList": True
                },
                {
                    "file": "filename"
                },
                {
                    "symbols": True
                }
            ]
        }
    }
}"""

{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "pot-name"
        },
        "units": "units",
        "atom_style": "style",
        "atom": {
            "element": "El",
            "symbol": "El1",
            "mass": 1.00
        },     
        "pair_style": {
            "type": "pair",
            "term": {
                "option": 1.0
            }
        },
        "pair_coeff": {
            "interaction": {
                "element": [
                    "El",
                    "El"
                ]
            },
            "term": [
                {
                    "option": 1.0
                },
                {
                    "symbolsList": True
                },
                {
                    "file": "filename"
                },
                {
                    "symbols": True
                }
            ]
        }
    }
}


###1.2 Examples

__This section contains example potential instance data models and code showing how the Potential class can be used to access and represent the data.__

###1.2.1 Simple style potential, i.e. Lennard-Jones.

__Potential instance data model file generator.  NOT A REAL POTENTIAL! ONLY FOR DEMO!__

In [4]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
f = open('lj-example.json', 'w')
f.write("""{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "lj-example"
        },
        "units": "lj",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "He"
            },
            {
                "element": "Ar"
            }
        ],
        "pair_style": {
            "type": "lj/cut",
            "term": {
                "option": 10.0
            }
        },
        "pair_coeff": [
            {
                "interaction": {
                    "element": [
                        "He",
                        "He"
                    ]
                },
                "term": {
                    "option": "1.0 1.0"
                }
            },
            {
                "interaction": {
                    "element": [
                        "Ar",
                        "Ar"
                    ]
                },
                "term": {
                    "option": "2.0 2.0"
                }
            },
            {
                "interaction": {
                    "element": [
                        "He",
                        "Ar"
                    ]
                },
                "term": {
                    "option": "1.0 2.0"
                }
            }            
        ]
    }
}""")
f.close()      

__With the AtomMan package, this file can be easily read in creating a object instance.__

In [5]:
pot = atomman.lammps.Potential('lj-example.json')
print 'Potential name is:', pot

Potential name is: lj-example


__The units and atom_style can be easily retrieved using appropriately named methods:__

In [6]:
print 'units', pot.units()
print 'atom_style', pot.atom_style()

units lj
atom_style atomic


__The elements(), symbols() and masses() methods return lists of the associated values.  elements() and masses() also can take arguments to return a specific value or list of values associated with a particular element symbol or list of symbols.__

In [7]:
symbols = pot.symbols()
symbol_list = [symbols[0], symbols[0], symbols[1], symbols[1], symbols[0], symbols[1]]
print 'Default method calls:'
print 'symbols =',  symbols
print 'elements =', pot.elements()
print 'masses =',   pot.masses()
print

print 'Values using a single specified element symbol:'
for i in xrange(len(symbols)):
    print 'Mass of', pot.elements(symbols[i]), 'is', pot.masses(symbols[i])
print

print 'Values using the symbol list', symbol_list
print 'elements =', pot.elements(symbol_list)
print 'masses =',   pot.masses(symbol_list)

Default method calls:
symbols = [u'He', u'Ar']
elements = [u'He', u'Ar']
masses = [4.002602, 39.948]

Values using a single specified element symbol:
Mass of He is 4.002602
Mass of Ar is 39.948

Values using the symbol list [u'He', u'He', u'Ar', u'Ar', u'He', u'Ar']
elements = [u'He', u'He', u'Ar', u'Ar', u'He', u'Ar']
masses = [4.002602, 4.002602, 39.948, 39.948, 4.002602, 39.948]


__Finally, a method pair_info() generates the mass, pair_style and pair_coeff lines__ 

In [8]:
print 'Default pair_info:'
print pot.pair_info()
print
print 'pair_info with one element symbol specified:'
print pot.pair_info(symbols[0])
print

print 'pair_info using the symbol list', symbol_list
print pot.pair_info(symbol_list)

Default pair_info:
mass 1 4.002602
mass 2 39.948000

pair_style lj/cut 10.0
pair_coeff 1 1 1.0 1.0
pair_coeff 2 2 2.0 2.0
pair_coeff 1 2 1.0 2.0


pair_info with one element symbol specified:
mass 1 4.002602

pair_style lj/cut 10.0
pair_coeff 1 1 1.0 1.0


pair_info using the symbol list [u'He', u'He', u'Ar', u'Ar', u'He', u'Ar']
mass 1 4.002602
mass 2 4.002602
mass 3 39.948000
mass 4 39.948000
mass 5 4.002602
mass 6 39.948000

pair_style lj/cut 10.0
pair_coeff 1 1 1.0 1.0
pair_coeff 1 2 1.0 1.0
pair_coeff 1 5 1.0 1.0
pair_coeff 2 2 1.0 1.0
pair_coeff 2 5 1.0 1.0
pair_coeff 5 5 1.0 1.0
pair_coeff 3 3 2.0 2.0
pair_coeff 3 4 2.0 2.0
pair_coeff 3 6 2.0 2.0
pair_coeff 4 4 2.0 2.0
pair_coeff 4 6 2.0 2.0
pair_coeff 6 6 2.0 2.0
pair_coeff 1 3 1.0 2.0
pair_coeff 1 4 1.0 2.0
pair_coeff 1 6 1.0 2.0
pair_coeff 2 3 1.0 2.0
pair_coeff 2 4 1.0 2.0
pair_coeff 2 6 1.0 2.0
pair_coeff 3 5 1.0 2.0
pair_coeff 4 5 1.0 2.0
pair_coeff 5 6 1.0 2.0



###1.2.2 Many-body potential with an associated potential parameter file, i.e. eam/alloy.

__Potential instance data model file generator.  NOT A REAL POTENTIAL! ONLY FOR DEMO!__

In [9]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
f = open('eam-alloy-example.json', 'w')
f.write("""{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "eam-alloy-example"
        },
        "units": "metal",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "Ni",
                "mass": 58.6934
            },
            {
                "element": "Al",
                "mass": 26.981539
            },
            {
                "element": "Co",
                "mass": 58.9332
            }
        ],
        "pair_style": {
            "type": "eam/alloy"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "file.eam.alloy"
                },
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")
f.close()      

__With the AtomMan package, this file can be easily read in creating a object instance.__

In [10]:
pot = atomman.lammps.Potential('eam-alloy-example.json')
print 'Potential name is:', pot

Potential name is: eam-alloy-example


__The units and atom_style can be easily retrieved using appropriately named methods:__

In [11]:
print 'units', pot.units()
print 'atom_style', pot.atom_style()

units metal
atom_style atomic


__The elements(), symbols() and masses() methods return lists of the associated values.  elements() and masses() also can take arguments to return a specific value or list of values associated with a particular element symbol or list of symbols.__

In [12]:
symbols = pot.symbols()
symbol_list = [symbols[0], symbols[0], symbols[1], symbols[1], symbols[0], symbols[1]]
print 'Default method calls:'
print 'symbols =',  symbols
print 'elements =', pot.elements()
print 'masses =',   pot.masses()
print

print 'Values using a single specified element symbol:'
for i in xrange(len(symbols)):
    print 'Mass of', pot.elements(symbols[i]), 'is', pot.masses(symbols[i])
print

print 'Values using the symbol list', symbol_list
print 'elements =', pot.elements(symbol_list)
print 'masses =',   pot.masses(symbol_list)

Default method calls:
symbols = [u'Ni', u'Al', u'Co']
elements = [u'Ni', u'Al', u'Co']
masses = [58.6934, 26.981539, 58.9332]

Values using a single specified element symbol:
Mass of Ni is 58.6934
Mass of Al is 26.981539
Mass of Co is 58.9332

Values using the symbol list [u'Ni', u'Ni', u'Al', u'Al', u'Ni', u'Al']
elements = [u'Ni', u'Ni', u'Al', u'Al', u'Ni', u'Al']
masses = [58.6934, 58.6934, 26.981539, 26.981539, 58.6934, 26.981539]


__Finally, a method pair_info() generates the mass, pair_style and pair_coeff lines__ 

In [13]:
print 'Default pair_info:'
print pot.pair_info()
print
print 'pair_info with one element symbol specified:'
print pot.pair_info(symbols[0])
print

print 'pair_info using the symbol list', symbol_list
print pot.pair_info(symbol_list)

Default pair_info:
mass 1 58.693400
mass 2 26.981539
mass 3 58.933200

pair_style eam/alloy
pair_coeff * * file.eam.alloy Ni Al Co


pair_info with one element symbol specified:
mass 1 58.693400

pair_style eam/alloy
pair_coeff * * file.eam.alloy Ni


pair_info using the symbol list [u'Ni', u'Ni', u'Al', u'Al', u'Ni', u'Al']
mass 1 58.693400
mass 2 58.693400
mass 3 26.981539
mass 4 26.981539
mass 5 58.693400
mass 6 26.981539

pair_style eam/alloy
pair_coeff * * file.eam.alloy Ni Ni Al Al Ni Al

